In [1]:
import os
import pickle
import platform
import random

import numpy as np
import torch
from transformer_lens import HookedTransformer

torch.set_grad_enabled(False)

SEED = 42

In [2]:
def is_mps_available() -> bool:
    """
    Safely check if MPS (Metal Performance Shaders) is available.
    """
    if platform.system() != "Darwin":  # MPS is only available on macOS
        return False

    # Check if the current PyTorch version has MPS support
    if not hasattr(torch, "backends") or not hasattr(torch.backends, "mps"):
        return False

    return torch.backends.mps.is_available()


def set_all_seeds(
    seed: int, deterministic: bool = True, warn_only: bool = False
) -> None:
    """
    Set all seeds and deterministic flags for reproducibility.

    Args:
        seed (int): The seed value to use for all random number generators
        deterministic (bool): Whether to enforce deterministic behavior
        warn_only (bool): If True, warning instead of error when deterministic
                         operations aren't supported
    """
    # Python RNG
    random.seed(seed)

    # NumPy RNG
    np.random.seed(seed)

    # PyTorch RNGs
    torch.manual_seed(seed)

    # Handle CUDA devices
    if torch.cuda.is_available():
        try:
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)  # for multi-GPU
        except Exception as e:
            print(f"Warning: Could not set CUDA seeds: {str(e)}")

    # Handle MPS device (Apple Silicon)
    if is_mps_available():
        try:
            torch.mps.manual_seed(seed)
        except Exception as e:
            print(f"Warning: Could not set MPS seed: {str(e)}")

    # Environment variables
    os.environ["PYTHONHASHSEED"] = str(seed)

    if deterministic:
        try:
            # CUDA deterministic behavior
            if torch.cuda.is_available():
                os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
                torch.backends.cudnn.deterministic = True
                torch.backends.cudnn.benchmark = False

            # Set deterministic algorithms
            torch.use_deterministic_algorithms(True, warn_only=warn_only)

        except Exception as e:
            msg = f"Warning: Could not enable deterministic mode. Error: {str(e)}"
            if not warn_only:
                raise RuntimeError(msg)
            print(msg)


def get_device() -> torch.device:
    """
    Get the most appropriate PyTorch device available.

    Returns:
        torch.device: The preferred device (CUDA > MPS > CPU)
    """
    if torch.cuda.is_available():
        return torch.device("cuda")
    if is_mps_available():
        return torch.device("mps")
    return torch.device("cpu")

In [4]:
set_all_seeds(SEED, warn_only=True)
device = get_device()
model_name = "gpt2"
model: HookedTransformer = HookedTransformer.from_pretrained(
    model_name, device=device
).eval()

Loaded pretrained model gpt2 into HookedTransformer


In [5]:
bank = {
    "city-country": {
        "bank": {
            "Bangkok": "Thailand",
            "Beijing": "China",
            "Buenos Aires": "Argentina",
            "Cape Town": "South Africa",
            "Hong Kong": "China",
            "Kuala Lumpur": "Malaysia",
            "Los Angeles": "United States",
            "Mexico City": "Mexico",
            "New Delhi": "India",
            "New York City": "United States",
            "Paris": "France",
            "Rio de Janeiro": "Brazil",
            "Rome": "Italy",
            "San Francisco": "United States",
            "St. Petersburg": "Russia",
            "Sydney": "Australia",
            "Tokyo": "Japan",
            "Toronto": "Canada",
        },
        "prompt": "%s is a city in the country of",
    },
    "city-continent": {
        "bank": {
            "Bangkok": "Asia",
            "Beijing": "Asia",
            "Buenos Aires": "South America",
            "Cape Town": "Africa",
            "Hong Kong": "Asia",
            "Kuala Lumpur": "Asia",
            "Los Angeles": "North America",
            "Mexico City": "North America",
            "New Delhi": "Asia",
            "New York City": "North America",
            "Paris": "Europe",
            "Rio de Janeiro": "South America",
            "Rome": "Europe",
            "San Francisco": "North America",
            "St. Petersburg": "Europe",
            "Sydney": "Oceania",
            "Tokyo": "Asia",
            "Toronto": "North America",
        },
        "prompt": "%s is a city in the continent of",
    },
    "city-language": {
        "bank": {
            "Bangkok": "Thai",
            "Beijing": "Chinese",
            "Buenos Aires": "Spanish",
            "Cape Town": "Afrikaans",
            "Kuala Lumpur": "Malay",
            "Los Angeles": "English",
            "Mexico City": "Spanish",
            "New Delhi": "Hindi",
            "New York City": "English",
            "Paris": "French",
            "Rio de Janeiro": "Portuguese",
            "Rome": "Italian",
            "San Francisco": "English",
            "St. Petersburg": "Russian",
            "Sydney": "English",
            "Tokyo": "Japanese",
            "Toronto": "English",
        },
        "prompt": "%s is a city where the language spoken is",
    },
    "occupation-duty": {
        "bank": {
            "to plan and design the construction of buildings": "architect",
            "to represent clients in court during trial": "lawyer",
            "to diagnose and treat issues related to the teeth": "dentist",
            "to create clothing and accessories according to current trends": "fashion designer",
            "to report news and information": "journalist",
            "to educate students in a classroom setting": "teacher",
            "to capture images using cameras": "photographer",
            "to lead and guide a team of representatives, employees, or workers": "manager",
            "to code, test, and maintain computer programs and applications": "software developer",
            "to prepare and cook food in a professional kitchen": "chef",
            "to care for patients in a medical setting and assist doctors": "nurse",
            "to enforce laws and protect citizens from crime": "police officer",
            "to repair and maintain vehicles and machinery in a workshop, garage, or factory": "mechanic",
            "to conduct research and experiments in a laboratory setting": "scientist",
            "to create visual art using various mediums, such as paint, clay, or digital tools": "artist",
            "to play musical instruments and perform for audiences in various settings": "musician",
        },
        "prompt": "My duties are %s; I am a",
    },
    "object-color": {
        "bank": {
            "apple": "red",
            "banana": "yellow",
            "carrot": "orange",
            "grape": "purple",
            "lemon": "yellow",
            "lime": "green",
            "orange": "orange",
            "pear": "green",
            "strawberry": "red",
            "tomato": "red",
            "blueberry": "blue",
            "cherry": "red",
            "eggplant": "purple",
            "kiwi": "brown",
            "peach": "orange",
            "plum": "purple",
            "watermelon": "green",
            "avocado": "green",
        },
        "prompt": "The color of the %s is usually",
    },
    "object-size": {
        "bank": {
            "diamond": "millimeter",
            "bamboo": "meter",
            "axe": "meter",
            "gopher": "centimeter",
            "saffron": "millimeter",
            "lime": "centimeter",
            "turmeric": "centimeter",
            "lion": "meter",
            "violet": "centimeter",
            "starfish": "centimeter",
            "charcoal": "centimeter",
            "turquoise": "centimeter",
            "flamingo": "meter",
            "pig": "meter",
            "cornmeal": "centimeter",
            "blackberry": "centimeter",
        },
        "prompt": 'Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of %s is commonly expressed in "',
    },
}


In [6]:
def make_prompt(entity: str = "city", target: str = "country", num_prompts: int = 30):
    pair = "{entity}-{target}".format(entity=entity, target=target)
    data = bank[pair]
    pairs = data["bank"]
    prompt = data["prompt"]
    cities = list(pairs.keys())

    # Generate N 3-shot prompts: "A is a city in the country of B. C is a city in the country of D. E is a city in the country of " ground_truth = F
    prompts = []
    for i in range(num_prompts):
        # Randomly sample 3 cities from the list + 1 ground truth city to be queried (all different)
        cities_sampled = torch.randperm(len(cities))[:4]
        countries_sampled = [pairs[cities[i]] for i in cities_sampled]
        ground_truth = countries_sampled[-1]

        # Generate the prompt
        prompt_instance = (
            prompt % cities[cities_sampled[0]]
            + " "
            + countries_sampled[0]
            + ". "
            + prompt % cities[cities_sampled[1]]
            + " "
            + countries_sampled[1]
            + ". "
            + prompt % cities[cities_sampled[2]]
            + " "
            + countries_sampled[2]
            + ". "
            + prompt % cities[cities_sampled[3]]
        )

        prompts.append((prompt_instance, ground_truth))
    return prompts

In [7]:
# city - country
city_country_prompts = make_prompt(entity="city", target="country", num_prompts=100)

# city - continent
city_continent_prompts = make_prompt(entity="city", target="continent", num_prompts=100)

# city - language
city_language_prompts = make_prompt(entity="city", target="language", num_prompts=100)

# duty - occupation
duty_occupation_prompts = make_prompt(
    entity="occupation", target="duty", num_prompts=100
)

# object - color
object_color_prompts = make_prompt(entity="object", target="color", num_prompts=100)

# object - size
object_size_prompts = make_prompt(entity="object", target="size", num_prompts=100)

In [8]:
# Print some samples
print(city_country_prompts[0])
print(city_continent_prompts[0])
print(city_language_prompts[0])
print(duty_occupation_prompts[0])
print(object_color_prompts[0])
print(object_size_prompts[0])

('Los Angeles is a city in the country of United States. Sydney is a city in the country of Australia. St. Petersburg is a city in the country of Russia. Mexico City is a city in the country of', 'Mexico')
('Tokyo is a city in the continent of Asia. Kuala Lumpur is a city in the continent of Asia. New Delhi is a city in the continent of Asia. Beijing is a city in the continent of', 'Asia')
('Toronto is a city where the language spoken is English. Bangkok is a city where the language spoken is Thai. Buenos Aires is a city where the language spoken is Spanish. New York City is a city where the language spoken is', 'English')
('My duties are to diagnose and treat issues related to the teeth; I am a dentist. My duties are to represent clients in court during trial; I am a lawyer. My duties are to code, test, and maintain computer programs and applications; I am a software developer. My duties are to enforce laws and protect citizens from crime; I am a', 'police officer')
('The color of the

In [9]:
# model.generate(
#     "She is living in Rome, therefore her country of residence is Italy. She is living in New Delhi, therefore her country of residence is India. She is living in Washington, therefore her country of residence is",
#     max_new_tokens=5,
#     temperature=1.0,
#     prepend_bos=True,
# )  # This will print the output of the model

In [10]:
city_country_correct = 0
city_country_retained = []
for prompt, gt in city_country_prompts:
    print(prompt)
    generation = model.generate(
        prompt, max_new_tokens=1, temperature=0.0, prepend_bos=True
    )
    print(generation)

    print("Ground truth: ", gt)
    print("\n\n")
    print("---------------------------------------------------")
    if gt.startswith(generation[len(prompt) :].strip()):
        city_country_correct += 1
        city_country_retained.append((prompt, gt))

print("Accuracy: ", city_country_correct / len(city_country_prompts))

Los Angeles is a city in the country of United States. Sydney is a city in the country of Australia. St. Petersburg is a city in the country of Russia. Mexico City is a city in the country of


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/1 [00:00<?, ?it/s]

Los Angeles is a city in the country of United States. Sydney is a city in the country of Australia. St. Petersburg is a city in the country of Russia. Mexico City is a city in the country of Mexico
Ground truth:  Mexico



---------------------------------------------------
San Francisco is a city in the country of United States. Hong Kong is a city in the country of China. Beijing is a city in the country of China. Mexico City is a city in the country of


  0%|          | 0/1 [00:00<?, ?it/s]

...

Sydney is a city in the country of Australia. Los Angeles is a city in the country of United States. Mexico City is a city in the country of Mexico. Cape Town is a city in the country of South
Ground truth:  South Africa



---------------------------------------------------
Rio de Janeiro is a city in the country of Brazil. Paris is a city in the country of France. New Delhi is a city in the country of India. New York City is a city in the country of


  0%|          | 0/1 [00:00<?, ?it/s]

Rio de Janeiro is a city in the country of Brazil. Paris is a city in the country of France. New Delhi is a city in the country of India. New York City is a city in the country of New
Ground truth:  United States



---------------------------------------------------
Los Angeles is a city in the country of United States. New Delhi is a city in the country of India. Buenos Aires is a city in the country of Argentina. Paris is a city in the country of


  0%|          | 0/1 [00:00<?, ?it/s]

Los Angeles is a city in the country of United States. New Delhi is a city in the country of India. Buenos Aires is a city in the country of Argentina. Paris is a city in the country of France
Ground truth:  France



---------------------------------------------------
Accuracy:  0.71


In [11]:
city_continent_correct = 0


city_continent_retained = []
for prompt, gt in city_continent_prompts:
    print(prompt)
    generation = model.generate(
        prompt, max_new_tokens=1, temperature=0.0, prepend_bos=True
    )
    print(generation)
    print("Ground truth: ", gt)
    print("\n\n")
    print("---------------------------------------------------")
    if gt.startswith(generation[len(prompt) :].strip()):
        city_continent_correct += 1
        city_continent_retained.append((prompt, gt))

print("Accuracy: ", city_continent_correct / len(city_continent_prompts))

Tokyo is a city in the continent of Asia. Kuala Lumpur is a city in the continent of Asia. New Delhi is a city in the continent of Asia. Beijing is a city in the continent of


  0%|          | 0/1 [00:00<?, ?it/s]

Tokyo is a city in the continent of Asia. Kuala Lumpur is a city in the continent of Asia. New Delhi is a city in the continent of Asia. Beijing is a city in the continent of Asia
Ground truth:  Asia



---------------------------------------------------
St. Petersburg is a city in the continent of Europe. Paris is a city in the continent of Europe. Buenos Aires is a city in the continent of South America. Hong Kong is a city in the continent of


  0%|          | 0/1 [00:00<?, ?it/s]

St. Petersburg is a city in the continent of Europe. Paris is a city in the continent of Europe. Buenos Aires is a city in the continent of South America. Hong Kong is a city in the continent of South
Ground truth:  Asia



---------------------------------------------------
Hong Kong is a city in the continent of Asia. New Delhi is a city in the continent of Asia. St. Petersburg is a city in the continent of Europe. San Francisco is a city in the continent of


...

Los Angeles is a city in the continent of North America. Sydney is a city in the continent of Oceania. Beijing is a city in the continent of Asia. Paris is a city in the continent of Europe
Ground truth:  Europe



---------------------------------------------------
Buenos Aires is a city in the continent of South America. Hong Kong is a city in the continent of Asia. Paris is a city in the continent of Europe. Rome is a city in the continent of


  0%|          | 0/1 [00:00<?, ?it/s]

Buenos Aires is a city in the continent of South America. Hong Kong is a city in the continent of Asia. Paris is a city in the continent of Europe. Rome is a city in the continent of Asia
Ground truth:  Europe



---------------------------------------------------
Bangkok is a city in the continent of Asia. New Delhi is a city in the continent of Asia. Mexico City is a city in the continent of North America. Kuala Lumpur is a city in the continent of


  0%|          | 0/1 [00:00<?, ?it/s]

Bangkok is a city in the continent of Asia. New Delhi is a city in the continent of Asia. Mexico City is a city in the continent of North America. Kuala Lumpur is a city in the continent of Asia
Ground truth:  Asia



---------------------------------------------------
Accuracy:  0.3


In [12]:
city_language_correct = 0
city_language_retained = []
for prompt, gt in city_language_prompts:
    print(prompt)
    generation = model.generate(
        prompt, max_new_tokens=1, temperature=0.0, prepend_bos=True
    )
    print(generation)
    print("Ground truth: ", gt)
    print("\n\n")
    print("---------------------------------------------------")
    if gt.startswith(generation[len(prompt) :].strip()):
        city_language_correct += 1
        city_language_retained.append((prompt, gt))

print("Accuracy: ", city_language_correct / len(city_language_prompts))

Toronto is a city where the language spoken is English. Bangkok is a city where the language spoken is Thai. Buenos Aires is a city where the language spoken is Spanish. New York City is a city where the language spoken is


  0%|          | 0/1 [00:00<?, ?it/s]

Toronto is a city where the language spoken is English. Bangkok is a city where the language spoken is Thai. Buenos Aires is a city where the language spoken is Spanish. New York City is a city where the language spoken is English
Ground truth:  English



---------------------------------------------------
Buenos Aires is a city where the language spoken is Spanish. Bangkok is a city where the language spoken is Thai. Rio de Janeiro is a city where the language spoken is Portuguese. New Delhi is a city where the language spoken is


  0%|          | 0/1 [00:00<?, ?it/s]

Buenos Aires is a city where the language spoken is Spanish. Bangkok is a city where the language spoken is Thai. Rio de Janeiro is a city where the language spoken is Portuguese. New Delhi is a city where the language spoken is Hindi
Ground truth:  Hindi



---------------------------------------------------
Buenos Aires is a city where the language spoken is Spanish. Tokyo is a city where the language spoken is Japanese. New York City is a city where the language spoken is English. Cape Town is a city where the language spoken is


...

Sydney is a city where the language spoken is English. Mexico City is a city where the language spoken is Spanish. New York City is a city where the language spoken is English. Los Angeles is a city where the language spoken is Spanish
Ground truth:  English



---------------------------------------------------
Sydney is a city where the language spoken is English. Rio de Janeiro is a city where the language spoken is Portuguese. Toronto is a city where the language spoken is English. New York City is a city where the language spoken is


  0%|          | 0/1 [00:00<?, ?it/s]

Sydney is a city where the language spoken is English. Rio de Janeiro is a city where the language spoken is Portuguese. Toronto is a city where the language spoken is English. New York City is a city where the language spoken is English
Ground truth:  English



---------------------------------------------------
New York City is a city where the language spoken is English. New Delhi is a city where the language spoken is Hindi. San Francisco is a city where the language spoken is English. Cape Town is a city where the language spoken is


  0%|          | 0/1 [00:00<?, ?it/s]

New York City is a city where the language spoken is English. New Delhi is a city where the language spoken is Hindi. San Francisco is a city where the language spoken is English. Cape Town is a city where the language spoken is English
Ground truth:  Afrikaans



---------------------------------------------------
Accuracy:  0.71


In [13]:
duty_occupation_correct = 0
duty_occupation_retained = []
for prompt, gt in duty_occupation_prompts:
    print(prompt)
    generation = model.generate(
        prompt, max_new_tokens=1, temperature=0.0, prepend_bos=True
    )
    print(generation)
    print("Ground truth: ", gt)
    print("\n\n")
    print("---------------------------------------------------")
    if gt.startswith(generation[len(prompt) :].strip()):
        duty_occupation_correct += 1
        duty_occupation_retained.append((prompt, gt))

print("Accuracy: ", duty_occupation_correct / len(duty_occupation_prompts))

My duties are to diagnose and treat issues related to the teeth; I am a dentist. My duties are to represent clients in court during trial; I am a lawyer. My duties are to code, test, and maintain computer programs and applications; I am a software developer. My duties are to enforce laws and protect citizens from crime; I am a


  0%|          | 0/1 [00:00<?, ?it/s]

My duties are to diagnose and treat issues related to the teeth; I am a dentist. My duties are to represent clients in court during trial; I am a lawyer. My duties are to code, test, and maintain computer programs and applications; I am a software developer. My duties are to enforce laws and protect citizens from crime; I am a lawyer
Ground truth:  police officer



---------------------------------------------------
My duties are to plan and design the construction of buildings; I am a architect. My duties are to diagnose and treat issues related to the teeth; I am a dentist. My duties are to care for patients in a medical setting and assist doctors; I am a nurse. My duties are to create clothing and accessories according to current trends; I am a


  0%|          | 0/1 [00:00<?, ?it/s]

My duties are to plan and design the construction of buildings; I am a architect. My duties are to diagnose and treat issues related to the teeth; I am a dentist. My duties are to care for patients in a medical setting and assist doctors; I am a nurse. My duties are to create clothing and accessories according to current trends; I am a nurse
Ground truth:  fashion designer



---------------------------------------------------
My duties are to create visual art using various mediums, such as paint, clay, or digital tools; I am a artist. My duties are to repair and maintain vehicles and machinery in a workshop, garage, or factory; I am a mechanic. My duties are to lead and guide a team of representatives, employees, or workers; I am a manager. My duties are to capture images using cameras; I am a


...

My duties are to create visual art using various mediums, such as paint, clay, or digital tools; I am a artist. My duties are to care for patients in a medical setting and assist doctors; I am a nurse. My duties are to repair and maintain vehicles and machinery in a workshop, garage, or factory; I am a mechanic. My duties are to diagnose and treat issues related to the teeth; I am a mechanic
Ground truth:  dentist



---------------------------------------------------
My duties are to diagnose and treat issues related to the teeth; I am a dentist. My duties are to care for patients in a medical setting and assist doctors; I am a nurse. My duties are to represent clients in court during trial; I am a lawyer. My duties are to educate students in a classroom setting; I am a


  0%|          | 0/1 [00:00<?, ?it/s]

My duties are to diagnose and treat issues related to the teeth; I am a dentist. My duties are to care for patients in a medical setting and assist doctors; I am a nurse. My duties are to represent clients in court during trial; I am a lawyer. My duties are to educate students in a classroom setting; I am a lawyer
Ground truth:  teacher



---------------------------------------------------
My duties are to diagnose and treat issues related to the teeth; I am a dentist. My duties are to lead and guide a team of representatives, employees, or workers; I am a manager. My duties are to represent clients in court during trial; I am a lawyer. My duties are to conduct research and experiments in a laboratory setting; I am a


  0%|          | 0/1 [00:00<?, ?it/s]

My duties are to diagnose and treat issues related to the teeth; I am a dentist. My duties are to lead and guide a team of representatives, employees, or workers; I am a manager. My duties are to represent clients in court during trial; I am a lawyer. My duties are to conduct research and experiments in a laboratory setting; I am a physician
Ground truth:  scientist



---------------------------------------------------
Accuracy:  0.29


In [14]:
object_color_correct = 0
object_color_retained = []
for prompt, gt in object_color_prompts:
    print(prompt)
    generation = model.generate(
        prompt, max_new_tokens=1, temperature=0.0, prepend_bos=True
    )
    print(generation)
    print("Ground truth: ", gt)
    print("\n\n")
    print("---------------------------------------------------")
    if gt.startswith(generation[len(prompt) :].strip()):
        object_color_correct += 1
        object_color_retained.append((prompt, gt))

print("Accuracy: ", object_color_correct / len(object_color_prompts))

The color of the cherry is usually red. The color of the tomato is usually red. The color of the peach is usually orange. The color of the apple is usually


  0%|          | 0/1 [00:00<?, ?it/s]

The color of the cherry is usually red. The color of the tomato is usually red. The color of the peach is usually orange. The color of the apple is usually orange
Ground truth:  red



---------------------------------------------------
The color of the banana is usually yellow. The color of the plum is usually purple. The color of the lemon is usually yellow. The color of the tomato is usually


  0%|          | 0/1 [00:00<?, ?it/s]

The color of the banana is usually yellow. The color of the plum is usually purple. The color of the lemon is usually yellow. The color of the tomato is usually orange
Ground truth:  red



---------------------------------------------------
The color of the strawberry is usually red. The color of the apple is usually red. The color of the orange is usually orange. The color of the grape is usually


...

The color of the banana is usually yellow. The color of the cherry is usually red. The color of the pear is usually green. The color of the lemon is usually blue
Ground truth:  yellow



---------------------------------------------------
The color of the orange is usually orange. The color of the kiwi is usually brown. The color of the cherry is usually red. The color of the tomato is usually


  0%|          | 0/1 [00:00<?, ?it/s]

The color of the orange is usually orange. The color of the kiwi is usually brown. The color of the cherry is usually red. The color of the tomato is usually green
Ground truth:  red



---------------------------------------------------
The color of the plum is usually purple. The color of the banana is usually yellow. The color of the blueberry is usually blue. The color of the lime is usually


  0%|          | 0/1 [00:00<?, ?it/s]

The color of the plum is usually purple. The color of the banana is usually yellow. The color of the blueberry is usually blue. The color of the lime is usually lime
Ground truth:  green



---------------------------------------------------
Accuracy:  0.31


In [15]:
object_size_correct = 0
object_size_retained = []
for prompt, gt in object_size_prompts:
    print(prompt)
    generation = model.generate(
        prompt, max_new_tokens=1, temperature=0.0, prepend_bos=True
    )
    print(generation)
    print("Ground truth: ", gt)
    print("\n\n")
    print("---------------------------------------------------")
    if gt.startswith(generation[len(prompt) :].strip()):
        object_size_correct += 1
        object_size_retained.append((prompt, gt))

print("Accuracy: ", object_size_correct / len(object_size_prompts))

Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of turquoise is commonly expressed in " centimeter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of starfish is commonly expressed in " centimeter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of cornmeal is commonly expressed in " centimeter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of pig is commonly expressed in "


  0%|          | 0/1 [00:00<?, ?it/s]

Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of turquoise is commonly expressed in " centimeter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of starfish is commonly expressed in " centimeter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of cornmeal is commonly expressed in " centimeter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of pig is commonly expressed in " cent
Ground truth:  meter



---------------------------------------------------
Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of cornmeal is commonly expressed in " centimeter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of flamingo is commonly expressed in " meter. Considering the 

  0%|          | 0/1 [00:00<?, ?it/s]

Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of cornmeal is commonly expressed in " centimeter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of flamingo is commonly expressed in " meter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of axe is commonly expressed in " meter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of turmeric is commonly expressed in " mill
Ground truth:  centimeter



---------------------------------------------------
Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of pig is commonly expressed in " meter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of turmeric is commonly expressed in " centimeter. Considering the following u

...

Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of bamboo is commonly expressed in " meter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of lion is commonly expressed in " meter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of violet is commonly expressed in " centimeter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of axe is commonly expressed in " cent
Ground truth:  meter



---------------------------------------------------
Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of lime is commonly expressed in " centimeter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of saffron is commonly expressed in " millimeter. Considering the following units " m

  0%|          | 0/1 [00:00<?, ?it/s]

Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of lime is commonly expressed in " centimeter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of saffron is commonly expressed in " millimeter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of blackberry is commonly expressed in " centimeter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of starfish is commonly expressed in " mill
Ground truth:  centimeter



---------------------------------------------------
Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of diamond is commonly expressed in " millimeter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of turmeric is commonly expressed in " centimeter. Consid

  0%|          | 0/1 [00:00<?, ?it/s]

Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of diamond is commonly expressed in " millimeter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of turmeric is commonly expressed in " centimeter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of turquoise is commonly expressed in " centimeter. Considering the following units " millimeter ", " centimeter ", " meter ", and " kilometer ", the size of blackberry is commonly expressed in " mill
Ground truth:  centimeter



---------------------------------------------------
Accuracy:  0.34


In [16]:
# Save in a pickle file the data {(entity-target): {accuracy : accuracy, retained: [(prompt, gt)], full_data: [(prompt, gt)]}}

data = {
    "city-country": {
        "accuracy": city_country_correct / len(city_country_prompts),
        "retained": city_country_retained,
        "full_data": city_country_prompts,
    },
    "city-continent": {
        "accuracy": city_continent_correct / len(city_continent_prompts),
        "retained": city_continent_retained,
        "full_data": city_continent_prompts,
    },
    "city-language": {
        "accuracy": city_language_correct / len(city_language_prompts),
        "retained": city_language_retained,
        "full_data": city_language_prompts,
    },
    "occupation-duty": {
        "accuracy": duty_occupation_correct / len(duty_occupation_prompts),
        "retained": duty_occupation_retained,
        "full_data": duty_occupation_prompts,
    },
    "object-color": {
        "accuracy": object_color_correct / len(object_color_prompts),
        "retained": object_color_retained,
        "full_data": object_color_prompts,
    },
    "object-size": {
        "accuracy": object_size_correct / len(object_size_prompts),
        "retained": object_size_retained,
        "full_data": object_size_prompts,
    },
}

with open(f"{model_name}_prompt_data.pkl", "wb") as f:
    pickle.dump(data, f)

In [18]:
# Print a recap of the results
for key, value in data.items():
    print(f"Accuracy for {key}: {value['accuracy']}")
    print("\n")
    print("---------------------------------------------------")

Accuracy for city-country: 0.71



---------------------------------------------------
Accuracy for city-continent: 0.3



---------------------------------------------------
Accuracy for city-language: 0.71



---------------------------------------------------
Accuracy for occupation-duty: 0.29



---------------------------------------------------
Accuracy for object-color: 0.31



---------------------------------------------------
Accuracy for object-size: 0.34



---------------------------------------------------
